In [1]:
import multiSyncPy as msp
from sktime.utils.plotting import plot_correlations
from multiSyncPy import synchrony_metrics as sm
from scipy.signal import hilbert
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from data_functions import get_ant_datafiles_info, preprocess_antcolony_data
from multisync_functions import get_multisync_metrics, get_multisync_statistics
from IPython.display import display
from SurrogateGenerator import segm_shuffle, trajectory_plot, get_nth_colony, surrogate_aaft, get_n_aafts, get_n_segm_shuffles
import warnings

ant_colonies = get_ant_datafiles_info()
ant_colonies


,recording_location,file,location,n_ants,species_name,enterance
0,indoor,dataset/ants/IndoorDataset/Seq0001Object10Imag...,china,10,japanese_arched_ant,0
1,indoor,dataset/ants/IndoorDataset/Seq0002Object10Imag...,china,10,japanese_arched_ant,0
2,indoor,dataset/ants/IndoorDataset/Seq0003Object10Imag...,china,10,japanese_arched_ant,0
3,indoor,dataset/ants/IndoorDataset/Seq0004Object10Imag...,china,10,japanese_arched_ant,0
4,indoor,dataset/ants/IndoorDataset/Seq0005Object10Imag...,china,10,japanese_arched_ant,0
5,outdoor,dataset/ants/OutdoorDataset/Seq0006Object21Ima...,russia,73,carpenter_ant,1
6,outdoor,dataset/ants/OutdoorDataset/Seq0007Object26Ima...,greece,162,little_black_ant,1
7,outdoor,dataset/ants/OutdoorDataset/Seq0008Object31Ima...,russia,133,carpenter_ant,1
8,outdoor,dataset/ants/OutdoorDataset/Seq0009Object50Ima...,russia,193,carpenter_ant,1
9,outdoor,dataset/ants/OutdoorDataset/Seq0010Object30Ima...,usa,101,little_black_ant,0


In [14]:
# get all metrics for all ant colonies on a certain target variable (ant x or y position) and compare indoor and outdoor ants
preprocessed_colonies = [preprocess_antcolony_data(colony) for colony in ant_colonies['file'].tolist()]
metrics = [get_multisync_metrics(colony, target_var='x', id_column='ant_id') for colony in preprocessed_colonies]
coherence, csd, symbolic_entropy, rho = zip(*metrics)

ant_colonies['coherence'] = coherence
ant_colonies['csd'] = csd
ant_colonies['symbolic_entropy'] = symbolic_entropy
ant_colonies['rho'] = rho

display("ORIGINAL")
display(ant_colonies.drop(columns=['file']))

'ORIGINAL'

,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic_entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.167,0.180,3.615,0.993
1,indoor,china,10,japanese_arched_ant,0,0.182,0.192,4.042,0.996
2,indoor,china,10,japanese_arched_ant,0,0.180,0.185,4.358,0.993
3,indoor,china,10,japanese_arched_ant,0,0.173,0.231,4.271,0.997
4,indoor,china,10,japanese_arched_ant,0,0.193,0.225,4.420,0.999
5,outdoor,russia,73,carpenter_ant,1,0.171,0.225,3.574,0.954
6,outdoor,greece,162,little_black_ant,1,0.173,0.216,3.569,0.953
7,outdoor,russia,133,carpenter_ant,1,0.172,0.204,3.740,0.995
8,outdoor,russia,193,carpenter_ant,1,0.198,0.298,3.211,0.918
9,outdoor,usa,101,little_black_ant,0,0.205,0.436,3.126,0.891


In [15]:
warnings.filterwarnings("ignore")

ants_surr1 = ant_colonies.copy()

example_surr_data = segm_shuffle(get_nth_colony(0), 'x', 'y','ant_id', 100)
ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
ant_dict = {ant_id: ants[ants['ant_id'] == ant_id] for ant_id in ants['ant_id'].unique()}
#print(f"Number of unique ants: {ants['ant_id'].nunique()}")

# make new columns 'coherence', 'csd', 'symbolic entropy', 'rho' in the ant_colonies dataframe
ants_surr1['coherence'] = None
ants_surr1['csd'] = None
ants_surr1['symbolic_entropy'] = None
ants_surr1['rho'] = None

for i in range(10):
    example_surr_data = get_nth_colony(i)
    ants = surrogate_aaft(example_surr_data, 'x', 'y', 'ant_id')
    metrics_colony = get_multisync_metrics(ants, target_var='surrogate_x', id_column='ant_id')
    coherence, csd, symbolic_entropy, rho = metrics_colony
    #metrics.append([coherence, csd, symbolic_entropy, rho])

    # in row i of ant_colonies set the value of column 'coherence' to coherence, etc.
    ants_surr1.loc[i, 'coherence'] = coherence
    ants_surr1.loc[i, 'csd'] = csd
    ants_surr1.loc[i, 'symbolic_entropy'] = symbolic_entropy
    ants_surr1.loc[i, 'rho'] = rho

display("SURROGATE")
display(ants_surr1.drop(columns=['file']))
original_metrics = ant_colonies.drop(columns=['file'])

'SURROGATE'

,recording_location,location,n_ants,species_name,enterance,coherence,csd,symbolic_entropy,rho
0,indoor,china,10,japanese_arched_ant,0,0.17,0.183,3.718,0.996
1,indoor,china,10,japanese_arched_ant,0,0.164,0.169,4.139,0.997
2,indoor,china,10,japanese_arched_ant,0,0.176,0.171,4.389,0.994
3,indoor,china,10,japanese_arched_ant,0,0.179,0.21,4.255,0.998
4,indoor,china,10,japanese_arched_ant,0,0.18,0.146,4.508,0.999
5,outdoor,russia,73,carpenter_ant,1,0.17,0.182,3.608,0.946
6,outdoor,greece,162,little_black_ant,1,0.155,0.171,3.747,0.957
7,outdoor,russia,133,carpenter_ant,1,0.169,0.197,3.798,0.991
8,outdoor,russia,193,carpenter_ant,1,0.186,0.264,3.228,0.958
9,outdoor,usa,101,little_black_ant,0,0.194,0.4,3.143,0.96


In [16]:
import pandas as pd
from scipy.stats import ttest_ind

metrics = ['coherence', 'csd', 'symbolic_entropy', 'rho']
results = []

for metric in metrics:
    original_metric = pd.to_numeric(original_metrics[metric], errors='coerce')
    surr_metric = pd.to_numeric(ants_surr1[metric], errors='coerce')
    t_stat, p_val = ttest_ind(original_metric.dropna(), surr_metric.dropna())
    results.append([metric, t_stat, p_val])

results_df = pd.DataFrame(results, columns=['Metric', 'T-Statistic', 'P-Value'])
results_df

,Metric,T-Statistic,P-Value
0,coherence,1.310800,0.206404
1,csd,0.885485,0.387572
2,symbolic_entropy,-0.292871,0.772970
3,rho,-0.766870,0.453100
